5.1 Simulated Data

Here is my own implementation based on the paper.

In [5]:
import numpy as np

def bvls_coordinate_descent_optimized(X, y, l, u, kappa=None, eps_kkt=1e-6, max_iter=10000):

    X = np.asarray(X, dtype=float)
    y = np.asarray(y, dtype=float)
    l = np.asarray(l, dtype=float)
    u = np.asarray(u, dtype=float)

    n, p = X.shape
    if kappa is None:
        kappa = min(n, p)


    # Initialization
    beta = np.where(np.abs(l) < np.abs(u), l, u).copy()
    
    # Precompute
    X_norms = np.linalg.norm(X, axis=0)
    y_norm = np.linalg.norm(y)
    scale = max(np.max(X_norms)*y_norm, 1.0)
    col_sums_sq = np.sum(X*X, axis=0)
    
    r = y - X @ (beta)
    
    def partial_derivative(i):
        return -X[:, i] @ (r)
    
    def kkt_check():

        tol = eps_kkt * scale
        for i in range(p):
            g_i = partial_derivative(i)
            if beta[i] <= l[i] + 1e-15:
                if g_i < -tol:
                    return False
            elif beta[i] >= u[i] - 1e-15:
                if g_i > tol:
                    return False
            else:
                if abs(g_i) > tol:
                    return False
        return True
    
    
    def compute_violations():
        tol = eps_kkt * scale
        deltas = np.zeros(p)
        for i in range(p):
            g_i = partial_derivative(i)
            neg_part = -max(-g_i, 0) if beta[i] < u[i] - 1e-15 else 0
            pos_part = max(g_i, 0) if beta[i] > l[i] + 1e-15 else 0
            delta = neg_part + pos_part
            deltas[i] = delta if delta > tol else 0.0
        return deltas
    
    

    def coordinate_update(i):

        Xi = X[:, i]
        denom = col_sums_sq[i]

        if denom < 1e-20:
            return beta[i]
        new_beta_i = Xi @ (r) / denom + beta[i]

        new_beta_i = min(max(new_beta_i, l[i]), u[i])
        
        if new_beta_i != beta[i]:
            diff = beta[i] - new_beta_i

            r[:] = r + Xi * diff
            beta[i] = new_beta_i
        return beta[i]
    

    S = np.zeros(p, dtype=bool)
    iteration = 0
    while iteration < max_iter:
        iteration += 1

        if kkt_check():
            break

        deltas = compute_violations()
        violating_inds = np.argsort(-deltas)
        
        count_added = 0
        for idx in violating_inds:
            if deltas[idx] > (eps_kkt * scale) and not S[idx]:
                S[idx] = True
                count_added += 1
                if count_added == kappa:
                    break
        
        if count_added == 0 and not np.all(S):
            S[:] = True

        if kkt_check():
            break

        S_indices = np.where(S)[0]

        inner_converged = False
        for _ in range(100): 
            old_beta_S = beta[S_indices].copy()
            for i in S_indices:
                coordinate_update(i)

            if np.linalg.norm(beta[S_indices] - old_beta_S) < 1e-12:
                inner_converged = True
                break

        if not inner_converged:
            A_mask = (S & (beta > l+1e-15) & (beta < u-1e-15))
            A_indices = np.where(A_mask)[0]
            for _ in range(100):
                old_beta_A = beta[A_indices].copy()
                for i in A_indices:
                    coordinate_update(i)

                if np.linalg.norm(beta[A_indices] - old_beta_A) < 1e-12:
                    break

    return beta

Below are the adelie and MOSEK solver

In [19]:
import numpy as np
import time
import mosek
from adelie.solver import bvls
import matplotlib.pyplot as plt
from math import inf

def mosek_nnls(X, y):

    Q = 2 * X.T @ X
    c = -2 * X.T @ y

    # For symmetry
    Q = 0.5 * (Q + Q.T)
    Q += 1e-8 * np.eye(X.shape[1])

    n = X.shape[1]

    with mosek.Env() as env:
        with env.Task() as task:
            task.appendvars(n)
            for i in range(n):
                task.putvarbound(i, mosek.boundkey.lo, 0.0, +np.inf)

            task.putclist(range(n), c)

            qosubi, qosubj, qoval = [], [], []
            for i in range(n):
                for j in range(i+1):
                    val = Q[i, j]
                    if val != 0.0:
                        qosubi.append(i)
                        qosubj.append(j)
                        qoval.append(val)

            task.putqobj(qosubi, qosubj, qoval)
            task.putobjsense(mosek.objsense.minimize)

            task.optimize()

            xx = [0.0]*n
            task.getxx(mosek.soltype.itr, xx)
    return np.array(xx)

def adelie_nnls(X, y):

    n_features = X.shape[1]
    lower_bounds = np.zeros(n_features)
    upper_bounds = np.full(n_features, np.inf)

    solution = bvls(X, y, lower_bounds, upper_bounds)
    return solution.beta


def bvls_nnls(X, y):
    
    n_features = X.shape[1]
    l = np.zeros(n_features)
    u = np.full(n_features, np.inf)

    solution = bvls_coordinate_descent_optimized(X, y, l, u, kappa=None, eps_kkt=1e-6)
    return solution

Now we can see the result.

In [ ]:
n_values = [100, 1000]  
p = 10000
sigma = 1.0
num_points = 20
mu_values = np.linspace(-3 * sigma, 3 * sigma, num_points)

results = {}

for n in n_values:

    np.random.seed(0)
    X = np.random.rand(n, p)
    mask = np.random.rand(n, p) < 0.8
    X[mask] = 0.0

    adelie_times = []
    mosek_times = []
    bvls_times = []

    adelie_objectives = []
    mosek_objectives = []
    bvls_objectives = []

    adelie_active_sizes = []
    mosek_active_sizes = []
    bvls_active_sizes = []

    for mu in mu_values:
        y = np.random.normal(mu, sigma, size=n)

        # Adelie solver
        start_time = time.time()
        x_adelie = adelie_nnls(X, y)
        adelie_times.append(time.time() - start_time)
        adelie_obj = 0.5 * np.sum((y - X.dot(x_adelie)) ** 2)
        adelie_objectives.append(adelie_obj)
        adelie_active_sizes.append(np.sum(x_adelie > 1e-12))

        # MOSEK solver
        start_time = time.time()
        x_mosek = mosek_nnls(X, y)
        mosek_times.append(time.time() - start_time)
        mosek_obj = 0.5 * np.sum((y - X.dot(x_mosek)) ** 2)
        mosek_objectives.append(mosek_obj)
        mosek_active_sizes.append(np.sum(x_mosek > 1e-12))

        # BVLS solver
        start_time = time.time()
        x_bvls = bvls_nnls(X, y) 
        bvls_times.append(time.time() - start_time)
        bvls_obj = 0.5 * np.sum((y - X.dot(x_bvls)) ** 2)
        bvls_objectives.append(bvls_obj)
        bvls_active_sizes.append(np.sum(x_bvls > 1e-12))

    results[n] = {
        "mu_values": mu_values,
        "adelie_times": adelie_times,
        "mosek_times": mosek_times,
        "bvls_times": bvls_times,
        "adelie_objectives": adelie_objectives,
        "mosek_objectives": mosek_objectives,
        "bvls_objectives": bvls_objectives,
        "adelie_actives": adelie_active_sizes,
        "mosek_actives": mosek_active_sizes,
        "bvls_actives": bvls_active_sizes,
    }

fig, axes = plt.subplots(2, 3, figsize=(12, 8))
axes = axes.flatten()

for i, n in enumerate(n_values):
    # Time
    ax = axes[i * 3 + 0]
    ax.plot(results[n]["mu_values"], results[n]["adelie_times"], marker="o", label="adelie")
    ax.plot(results[n]["mu_values"], results[n]["mosek_times"], marker="o", label="MOSEK")
    ax.plot(results[n]["mu_values"], results[n]["bvls_times"], marker="o", label="BVLS")
    ax.set_yscale("log")
    ax.set_xlabel(r"$\mu$")
    ax.set_ylabel("Time (s)")
    ax.legend()
    ax.set_title(f"n={n}, Time")

    # Objective 
    ax = axes[i * 3 + 1]
    ax.plot(results[n]["mu_values"], results[n]["adelie_objectives"], marker="o", label="adelie")
    ax.plot(results[n]["mu_values"], results[n]["mosek_objectives"], marker="o", label="MOSEK")
    ax.plot(results[n]["mu_values"], results[n]["bvls_objectives"], marker="o", label="BVLS")
    ax.set_xlabel(r"$\mu$")
    ax.set_ylabel(r"$\frac{1}{2}\|y - Xx\|_2^2$")
    ax.set_title(f"n={n}, Objective")

    # Active set
    ax = axes[i * 3 + 2]
    ax.plot(results[n]["mu_values"], results[n]["adelie_actives"], marker="o", label="adelie active size")

    ax.plot(results[n]["mu_values"], results[n]["bvls_actives"], marker="o", label="BVLS active size",color = '#2ca02c')
    ax.set_xlabel(r"$\mu$")
    ax.set_ylabel("Active set size")
    ax.set_title(f"n={n}, Active set size")
    ax.legend()

plt.tight_layout()
plt.show()

5.2 Deconvolution of spike trains on carbon monoxide levels

First we import data from UCI, then we preprocess the data so that it is the same as authors.

In [8]:
from ucimlrepo import fetch_ucirepo 

air_quality = fetch_ucirepo(id=360) 
raw_data = air_quality.data.features
raw_data = raw_data.loc[raw_data["CO(GT)"] != -200]

data = raw_data[0:304]
data

,Date,Time,CO(GT),PT08.S1(CO),NMHC(GT),C6H6(GT),PT08.S2(NMHC),NOx(GT),PT08.S3(NOx),NO2(GT),PT08.S4(NO2),PT08.S5(O3),T,RH,AH
0,3/10/2004,18:00:00,2.6,1360,150,11.9,1046,166,1056,113,1692,1268,13.6,48.9,0.7578
1,3/10/2004,19:00:00,2.0,1292,112,9.4,955,103,1174,92,1559,972,13.3,47.7,0.7255
2,3/10/2004,20:00:00,2.2,1402,88,9.0,939,131,1140,114,1555,1074,11.9,54.0,0.7502
3,3/10/2004,21:00:00,2.2,1376,80,9.2,948,172,1092,122,1584,1203,11.0,60.0,0.7867
4,3/10/2004,22:00:00,1.6,1272,51,6.5,836,131,1205,116,1490,1110,11.2,59.6,0.7888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,3/23/2004,19:00:00,2.7,1236,236,13.0,1084,131,854,112,1688,1059,11.7,57.1,0.7822
314,3/23/2004,20:00:00,3.3,1309,220,14.3,1126,166,812,128,1722,1205,11.5,58.2,0.7867
315,3/23/2004,21:00:00,1.7,1089,87,7.6,884,89,1006,101,1487,975,12.6,54.2,0.7881
316,3/23/2004,22:00:00,1.3,1000,81,5.3,776,81,1141,96,1375,896,12.0,54.9,0.7664


In [9]:
import pandas as pd

data['Datetime'] = pd.to_datetime(data['Date'] + ' ' + data['Time'], format='%m/%d/%Y %H:%M:%S')

start_time = data['Datetime'].min()
data['Relative_Time'] = (data['Datetime'] - start_time).dt.total_seconds()

max_time = data['Relative_Time'].max()
data['Normalized_Time'] = data['Relative_Time'] / max_time

print(data[['Datetime', 'Relative_Time', 'Normalized_Time']])

               Datetime  Relative_Time  Normalized_Time
0   2004-03-10 18:00:00            0.0         0.000000
1   2004-03-10 19:00:00         3600.0         0.003155
2   2004-03-10 20:00:00         7200.0         0.006309
3   2004-03-10 21:00:00        10800.0         0.009464
4   2004-03-10 22:00:00        14400.0         0.012618
..                  ...            ...              ...
313 2004-03-23 19:00:00      1126800.0         0.987382
314 2004-03-23 20:00:00      1130400.0         0.990536
315 2004-03-23 21:00:00      1134000.0         0.993691
316 2004-03-23 22:00:00      1137600.0         0.996845
317 2004-03-23 23:00:00      1141200.0         1.000000

[304 rows x 3 columns]


/var/folders/z7/0_03j2751_bft60b1yn28_bc0000gn/T/ipykernel_27138/697393803.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Datetime'] = pd.to_datetime(data['Date'] + ' ' + data['Time'], format='%m/%d/%Y %H:%M:%S')
/var/folders/z7/0_03j2751_bft60b1yn28_bc0000gn/T/ipykernel_27138/697393803.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Relative_Time'] = (data['Datetime'] - start_time).dt.total_seconds()
/var/folders/z7/0_03j2751_bft60b1yn28_bc0000gn/T/ipykernel_27138/697393803.py:9: Setti

Construct the feature matrix X

In [22]:
import numpy as np
t = data['Normalized_Time'].to_numpy()

p = 1000
tau = np.linspace(0, 1, p)
sigma = 2*data['Normalized_Time'].diff().dropna().median()

diff_matrix = t[:, None] - tau[None, :]
X = (1 / (np.sqrt(2 * np.pi) * sigma)) * np.exp(-(diff_matrix**2) / (2 * sigma**2))
y = data['CO(GT)'].values

print(X.shape, y.shape)

(304, 1000) (304,)


In [17]:
from fnnls import fnnls

def fnnls_nnls(X, y):

    return fnnls(X, y)[0]

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt

# Adelie
start = time.time()
x_adelie = adelie_nnls(X, y)
adelie_time = time.time() - start
y_hat_adelie = X @ x_adelie

# MOSEK
start = time.time()
x_mosek = mosek_nnls(X, y)
mosek_time = time.time() - start
y_hat_mosek = X @ x_mosek

# FNNLS
start = time.time()
x_fnnls = fnnls_nnls(X, y)
fnnls_time = time.time() - start
y_hat_fnnls = X @ x_fnnls

# BVLS
start = time.time()
x_bvls = bvls_nnls(X, y)
bvls_time = time.time() - start
y_hat_bvls = X @ x_bvls

# Plot Observed vs. NNLS Fits
plt.figure(figsize=(12, 6), dpi=1000)
plt.plot(t, y, 'k-', label='$y$ (observed)', linewidth=1)
plt.plot(t, y_hat_adelie, 'b--', label='$\hat{y}$ (Adelie)', linewidth=1)
plt.plot(t, y_hat_mosek, 'r-.', label='$\hat{y}$ (MOSEK)', linewidth=1)
plt.plot(t, y_hat_fnnls, 'g:', label='$\hat{y}$ (FNNLS)', linewidth=1)
plt.plot(t, y_hat_bvls, 'm-.', label='$\hat{y}$ (BVLS)', linewidth=1)
plt.xlabel('Relative time')
plt.ylabel('CO (mg/m^3)')
plt.title('CO Level: Observed vs NNLS Fits (Adelie, MOSEK, FNNLS, BVLS)')
plt.legend()
plt.show()

# Print Computation Times
print(f"Adelie solution time: {adelie_time:.4f} s")
print(f"MOSEK solution time: {mosek_time:.4f} s")
print(f"FNNLS solution time: {fnnls_time:.4f} s")
print(f"BVLS solution time: {bvls_time:.4f} s")

# Identify Top 30 Spikes from BVLS Solution
bvls_spike_indices = np.argsort(x_bvls)[-top_k:]
bvls_spike_times = tau[bvls_spike_indices]

plt.figure(figsize=(12, 3), dpi=1000)
plt.plot(tau, x_bvls, 'o-', markersize=2, label='BVLS coefficients')
plt.plot(bvls_spike_times, x_bvls[bvls_spike_indices], 'rx', label='Top 30 spikes (BVLS)', markersize=10)
plt.xlabel('tau')
plt.ylabel('Coefficient value')
plt.title('BVLS Coefficients and Identified Spikes')
plt.legend()
plt.show()